In [4]:
!pip3 install --upgrade --quiet langchain langchain-community langchain-openai chromadb 
!pip3 install --upgrade --quiet pypdf pandas streamlit python-dotenv
!pip3 install --upgrade --quiet transformers sentence-transformers langchain

In [1]:
# Import Langchain modules
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

# Other modules and packages
import os
import tempfile
import streamlit as st  
import pandas as pd
from dotenv import load_dotenv

c:\Users\RivyeschRanjan\Desktop\Research\rag_llm\myenv\Lib\site-packages\IPython\core\interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
from langchain_community.llms import Ollama

In [3]:
llm = Ollama(model='llama3.1:8b')

In [19]:
# llm.invoke("The first man on the moon was ...")

'...Neil Armstrong!\n\nOn July 20, 1969, Neil Armstrong became the first person to set foot on the lunar surface during the Apollo 11 mission. As he stepped off the lunar module Eagle and onto the moon\'s surface, he famously declared:\n\n"That\'s one small step for man, one giant leap for mankind."\n\nWould you like to know more about his historic achievement?'

In [38]:
# from dotenv import load_dotenv, find_dotenv
# import sys
# path = sys.path[1]+'/myenv'  #try .path[0] if 1 doesn't work
# load_dotenv(path)

False

In [11]:
loader = PyPDFLoader("Company_Profile.pdf")
pages = loader.load()
pages


[Document(metadata={'source': 'Company_Profile.pdf', 'page': 0}, page_content='INSIGHT\nCOMPANY BROCHURENISE\n+03 5635 3248\nwww.nise.com\nsupport@nise.my'),
 Document(metadata={'source': 'Company_Profile.pdf', 'page': 1}, page_content='About Us\nNiSE\nAcademyOur Offering\nMaintenance\n& Support\nServicesAbout Us\nRecognizing the crucial role of skills and knowledge in\nsustaining business solutions, our commitment begins\nwith offering extensive training services.\nSpecializing in MicroStrategy, NiSE Insight is powered by\na team with of business intelligence experience\nspanning two decades.+03 5635 3248\nwww.nise.com\nsupport@nise.my\nA-05-29, Sunway Geo Avenue, Jalan Lagoon\nSelatan, Sunway South Quay, Bandar Sunway,\n47500 Subang Jaya, Selangor Darul Ehsan\n4 32 1Established in\nOctober 2017Local \nServices Provider\n100%\nBumiputra95% \nTechnical Team'),
 Document(metadata={'source': 'Company_Profile.pdf', 'page': 2}, page_content='About Us\nNiSE\nAcademyOur Offering\nMaintenance

In [23]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=350,
                                            chunk_overlap=100,
                                            length_function=len,
                                            separators=["\n\n", "\n", " "])
chunks = text_splitter.split_documents(pages)

In [24]:
chunks

[Document(metadata={'source': 'Company_Profile.pdf', 'page': 0}, page_content='INSIGHT\nCOMPANY BROCHURENISE\n+03 5635 3248\nwww.nise.com\nsupport@nise.my'),
 Document(metadata={'source': 'Company_Profile.pdf', 'page': 1}, page_content='About Us\nNiSE\nAcademyOur Offering\nMaintenance\n& Support\nServicesAbout Us\nRecognizing the crucial role of skills and knowledge in\nsustaining business solutions, our commitment begins\nwith offering extensive training services.\nSpecializing in MicroStrategy, NiSE Insight is powered by\na team with of business intelligence experience'),
 Document(metadata={'source': 'Company_Profile.pdf', 'page': 1}, page_content='a team with of business intelligence experience\nspanning two decades.+03 5635 3248\nwww.nise.com\nsupport@nise.my\nA-05-29, Sunway Geo Avenue, Jalan Lagoon\nSelatan, Sunway South Quay, Bandar Sunway,\n47500 Subang Jaya, Selangor Darul Ehsan\n4 32 1Established in\nOctober 2017Local \nServices Provider\n100%\nBumiputra95% \nTechnical Team'

In [25]:
from langchain.embeddings import OllamaEmbeddings

def get_embedding_function():
    embeddings = OllamaEmbeddings(model="llama3.1:8b")
    return embeddings

embedding_function = get_embedding_function()

# llm = Ollama(model='llama3.1:8b')
 
# vectorstoredb = FAISS.load_local(faiss_store_path, embeddings, allow_dangerous_deserialization=True)
 
# retriever = vectorstoredb.as_retriever()

In [26]:
from langchain.evaluation import load_evaluator

evaluator = load_evaluator(evaluator="embedding_distance", 
                            embeddings=embedding_function)

# evaluator.evaluate_strings(prediction="Amsterdam", reference="coffeeshop")

In [55]:
# evaluator.evaluate_strings(prediction="Paris", reference="coffeeshop")

{'score': 0.42799206948277213}

In [27]:
import uuid

def create_vectorstore(chunks, embedding_function, vectorstore_path):

    # Create a list of unique ids for each document based on the content
    ids = [str(uuid.uuid5(uuid.NAMESPACE_DNS, doc.page_content)) for doc in chunks]
    
    # Ensure that only unique docs with unique ids are kept
    unique_ids = set()
    unique_chunks = []
    
    unique_chunks = [] 
    for chunk, id in zip(chunks, ids):     
        if id not in unique_ids:       
            unique_ids.add(id)
            unique_chunks.append(chunk) 

    # Create a new Chroma database from the documents
    vectorstore = Chroma.from_documents(documents=unique_chunks, 
                                        ids=list(unique_ids),
                                        embedding=embedding_function, 
                                        persist_directory = vectorstore_path)

    vectorstore.persist()
    
    return vectorstore

In [28]:
# Create vectorstore
vectorstore = create_vectorstore(chunks=chunks, 
                                 embedding_function=embedding_function, 
                                 vectorstore_path="vectorstore_test")

ValueError: Error raised by inference API HTTP code: 500, {"error":"model requires more system memory (5.9 GiB) than is available (5.5 GiB)"}

In [70]:
# Load vectorstore
vectorstore = Chroma(persist_directory="vectorstore_test", embedding_function=embedding_function)

In [71]:
# Create retriever and get relevant chunks
retriever = vectorstore.as_retriever(search_type="similarity")
relevant_chunks = retriever.invoke("How much was the employee EPF contribution in MYR?")
relevant_chunks

[Document(metadata={'page': 0, 'source': 'Remittance - 50-CR002539GT4PAY070322-339.pdf'}, page_content='SOCSO - Employee 28/02/22 -19.75 -19.75 MYR\nMalaysia Income Tax 28/02/22 -20.80 -20.80 MYR'),
 Document(metadata={'page': 0, 'source': 'Remittance - 50-CR002539GT4PAY070322-339.pdf'}, page_content='EPF - Employee 28/02/22 -333.00 -333.00 MYR\nSOCSO - Employee 28/02/22 -19.75 -19.75 MYR'),
 Document(metadata={'page': 0, 'source': 'Remittance - 50-CR002539GT4PAY070322-339.pdf'}, page_content='Malaysia Income Tax 28/02/22 -20.80 -20.80 MYR\n4,012.45 Payment Due MYR'),
 Document(metadata={'page': 0, 'source': 'Remittance - 50-CR002539GT4PAY070322-339.pdf'}, page_content='EIS - Employee 28/02/22 -7.90 -7.90 MYR\nEPF - Employee 28/02/22 -333.00 -333.00 MYR')]

In [73]:
# Prompt template
PROMPT_TEMPLATE = """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer
the question. If you don't know the answer, say that you
don't know. DON'T MAKE UP ANYTHING.

{context}

---

Answer the question based on the above context: {question}
"""

In [74]:
# Concatenate context text
context_text = "\n\n---\n\n".join([doc.page_content for doc in relevant_chunks])

# Create prompt
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, 
                                question="How much was the employee EPF contribution in MYR?")
print(prompt)

Human: 
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer
the question. If you don't know the answer, say that you
don't know. DON'T MAKE UP ANYTHING.

SOCSO - Employee 28/02/22 -19.75 -19.75 MYR
Malaysia Income Tax 28/02/22 -20.80 -20.80 MYR

---

EPF - Employee 28/02/22 -333.00 -333.00 MYR
SOCSO - Employee 28/02/22 -19.75 -19.75 MYR

---

Malaysia Income Tax 28/02/22 -20.80 -20.80 MYR
4,012.45 Payment Due MYR

---

EIS - Employee 28/02/22 -7.90 -7.90 MYR
EPF - Employee 28/02/22 -333.00 -333.00 MYR

---

Answer the question based on the above context: How much was the employee EPF contribution in MYR?



In [75]:
llm.invoke(prompt)

"According to the retrieved context, the employee's EPF contribution is -333.00 MYR."

In [93]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
            {"context": retriever | format_docs, "question": RunnablePassthrough()}
            | prompt_template
            | llm
        )
# rag_chain.invoke("What date was the EPF deduction made?")
rag_chain.invoke("Give me the name on the remittance, total pay, date of pay and the number of overtime hours the individual worked.")

'Unfortunately, I don\'t have that information in the provided context. The names of the individuals are not mentioned. However, I can tell you that the date concerned is 28/02/22. As for the total pay, it\'s difficult to determine without knowing which specific transactions (e.g., EPF, SOCSO, EIS) belong to this individual.\n\nThe only transaction related directly to an "Employee" is for EIS and SOCSO on 28/02/22, with a total of -19.75 MYR.'

In [78]:
class AnswerWithSources(BaseModel):
    """An answer to the question, with sources and reasoning."""
    answer: str = Field(description="Answer to question")
    sources: str = Field(description="Full direct text chunk from the context used to answer the question")
    reasoning: str = Field(description="Explain the reasoning of the answer based on the sources")

class ExtractedInfo(BaseModel):
    """Extracted information about the payslip"""
    name: AnswerWithSources
    total_pay: AnswerWithSources
    date: AnswerWithSources
    overtime: AnswerWithSources

# class ExtractedInfo(BaseModel):
#     """Extracted information about the research article"""
#     paper_title: AnswerWithSources
#     paper_summary: AnswerWithSources
#     publication_year: AnswerWithSources
#     paper_authors: AnswerWithSources

In [91]:
rag_chain = (
            {"context": retriever | format_docs, "question": RunnablePassthrough()}
            | prompt_template
            | llm.as_structured_llm(output_cls=ExtractedInfo)
            # | llm.with_structured_output(ExtractedInfo, strict=True)
        )

rag_chain.invoke("Give me the name on the remittance, total pay, date of pay and the number of overtime hours the individual worked.")

AttributeError: 'Ollama' object has no attribute 'as_structured_llm'